# Chapter 8: Dimensionality Reduction

## 1. Chapter Overview
**Goal:** Many Machine Learning problems involve thousands or even millions of features for each training instance. This makes training extremely slow and makes it harder to find a good solution. This problem is often referred to as the **Curse of Dimensionality**. In this chapter, we explore techniques to reduce the number of features (dimensions) while losing as little information as possible.

**Key Concepts:**
* **The Curse of Dimensionality:** Why high-dimensional space is sparse and dangerous.
* **Approaches:** Projection vs. Manifold Learning.
* **PCA (Principal Component Analysis):** Preserving variance, SVD, and reconstruction error.
* **Choosing the Right Dimensions:** The "Elbow" method and 95% variance rule.
* **PCA Variations:** Randomized PCA, Incremental PCA (for large datasets).
* **Kernel PCA:** Performing complex nonlinear projections.
* **LLE (Locally Linear Embedding):** A powerful manifold learning technique.

**Practical Skills:**
* Implementing PCA using Scikit-Learn to compress data.
* Visualizing high-dimensional data in 2D or 3D.
* Tuning hyperparameters for Kernel PCA.
* Using LLE to unroll a "Swiss Roll" dataset.

In [ ]:
# Setup
import sys
import sklearn
import numpy as np
import os
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

np.random.seed(42)
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

## 2. Theoretical Explanation (In-Depth)

### 1. The Curse of Dimensionality
We are used to living in 3 dimensions. It is very hard to intuit what happens in 1,000 dimensions. High-dimensional spaces behave very differently from the 2D/3D space we know.

**Sparseness:**
If you pick a random point in a unit square (2D), there is a 0.04% chance it is located near the border (extreme value). But in a 10,000-dimensional hypercube, the probability is greater than 99.999999%. This means high-dimensional datasets are at risk of being very sparse: most training instances are likely to be far away from each other. 

**Distance & Overfitting:**
Because instances are far apart, predictions become much less reliable than in lower dimensions, and the model is prone to **overfitting**. Theoretically, you could solve this by adding more training data to fill the gaps, but the number of instances required grows exponentially with the number of dimensions.

### 2. Main Approaches to Reduction

**A. Projection:**
In most real-world problems, training instances are not spread out uniformly across all dimensions. Many features are constant, while others are highly correlated. As a result, all training instances actually lie within (or close to) a much lower-dimensional *subspace*.
* *Analogy:* Imagine a 3D object (like a chair) casting a shadow on a 2D wall. If you deal with the shadow (2D), you lose some information (depth), but you keep the general shape. This is projection.

**B. Manifold Learning:**
Sometimes, projection is not enough. Consider the **Swiss Roll** dataset (a 2D plane rolled up into a 3D spiral). If you project this onto a 2D plane (squash it), layers will overlap and mix the colors/classes. What you really want to do is *unroll* the Swiss Roll.
* A **d-dimensional manifold** is a part of an n-dimensional space (where d < n) that locally resembles a d-dimensional hyperplane. 
* **Manifold Learning** relies on the assumption that real-world high-dimensional datasets actually lie close to a much lower-dimensional manifold.

### 3. PCA (Principal Component Analysis)
PCA is by far the most popular dimensionality reduction algorithm. It identifies the hyperplane that lies closest to the data and then projects the data onto it.

**Preserving Variance:**
Before projecting, you need to choose the right hyperplane. PCA selects the axis that preserves the maximum amount of variance (information). 
* If you project a long oval onto its short axis, you lose a lot of spread (variance). 
* If you project it onto its long axis, you keep most of the spread. 
PCA finds the axis that minimizes the mean squared distance between the original dataset and its projection.

**Principal Components:**
PCA finds the first axis (PC1) that accounts for the largest amount of variance. Then it finds a second axis (PC2), orthogonal to the first, that accounts for the largest amount of remaining variance, and so on.

**SVD (Singular Value Decomposition):**
How does PCA find these axes mathematically? It uses a standard matrix factorization technique called SVD. It can decompose the training set matrix $X$ into the dot product of three matrices: $U \cdot \Sigma \cdot V^T$. 
The matrix $V^T$ contains the unit vectors that define all the Principal Components.

### 4. Variations of PCA

**Randomized PCA:**
Instead of using the full SVD algorithm (which is slow: $O(m \times n^2) + O(n^3)$), Scikit-Learn uses a stochastic algorithm called Randomized PCA. It quickly finds an approximation of the first $d$ principal components. Its complexity is drastically lower: $O(m \times d^2) + O(d^3)$.

**Incremental PCA (IPCA):**
Standard PCA requires the whole dataset to fit in memory. For huge datasets, IPCA allows you to split the training set into mini-batches and feed the algorithm one mini-batch at a time.

**Kernel PCA (kPCA):**
Just like with SVMs, we can use the Kernel Trick to implicitly map instances into a very high-dimensional feature space (where they become linearly separable) and then perform PCA in that space. This is great for unrolling nonlinear datasets like the Swiss Roll.

### 5. LLE (Locally Linear Embedding)
LLE is a Manifold Learning technique that does not rely on projections. 
1. For each training instance $x^{(i)}$, it identifies its $k$ nearest neighbors.
2. It tries to reconstruct $x^{(i)}$ as a linear function of these neighbors.
3. It then finds a low-dimensional representation of the data that preserves these local relationships as much as possible.
LLE is very good at unrolling twisted manifolds where PCA would fail.

## 3. Code Reproduction

### 3.1 PCA for Data Compression
We will create a simple 3D dataset that is actually flat (shaped like a pancake) and project it down to 2D.

In [ ]:
# Generate a 3D dataset
np.random.seed(4)
m = 60
w1, w2 = 0.1, 0.3
noise = 0.1

angles = np.random.rand(m) * 3 * np.pi / 2 - 0.5
X = np.empty((m, 3))
X[:, 0] = np.cos(angles) + np.sin(angles)/2 + noise * np.random.randn(m) / 2
X[:, 1] = np.sin(angles) * 0.7 + noise * np.random.randn(m) / 2
X[:, 2] = X[:, 0] * w1 + X[:, 1] * w2 + noise * np.random.randn(m)

print("Original shape:", X.shape)

# Apply PCA to reduce to 2 dimensions
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X2D = pca.fit_transform(X)

print("Reduced shape:", X2D.shape)
print("Explained Variance Ratio:", pca.explained_variance_ratio_)

The `explained_variance_ratio_` tells us that the first dimension carries e.g., 84% of the dataset's variance, and the second carries 14%. We lost less than 2% of the information by dropping the 3rd dimension.

### 3.2 Choosing the Right Number of Dimensions (MNIST)
Instead of guessing `n_components`, we can plot the cumulative explained variance as a function of the number of dimensions.

In [ ]:
from sklearn.datasets import fetch_openml

# Fetch MNIST (this might take a while)
mnist = fetch_openml('mnist_784', version=1, as_frame=False)
X, y = mnist["data"], mnist["target"]
X_train, X_test = X[:60000], X[60000:]

# Compute PCA without reducing dimensionality yet to see full variance
pca = PCA()
pca.fit(X_train)
cumsum = np.cumsum(pca.explained_variance_ratio_)
d = np.argmax(cumsum >= 0.95) + 1

print(f"Dimensions required to preserve 95% variance: {d}")

# Plotting the Elbow Curve
plt.figure(figsize=(6, 4))
plt.plot(cumsum, linewidth=3)
plt.axis([0, 400, 0, 1])
plt.xlabel("Dimensions")
plt.ylabel("Explained Variance")
plt.plot([d, d], [0, 0.95], "k:")
plt.plot([0, d], [0.95, 0.95], "k:")
plt.plot(d, 0.95, "ko")
plt.grid(True)
plt.title("Explained Variance vs Dimensions")
plt.show()

### 3.3 Data Compression
We can compress the dataset to 154 dimensions (from 784) and then decompress it back to 784. The result won't be identical, but it should be very close.

In [ ]:
pca = PCA(n_components=0.95)
X_reduced = pca.fit_transform(X_train)
X_recovered = pca.inverse_transform(X_reduced)

# Plotting Original vs Compressed
def plot_digits(instances, images_per_row=5, **options):
    size = 28
    images_per_row = min(len(instances), images_per_row)
    images = [instance.reshape(size,size) for instance in instances]
    n_rows = (len(instances) - 1) // images_per_row + 1
    row_images = []
    n_empty = n_rows * images_per_row - len(instances)
    images.append(np.zeros((size, size * n_empty)))
    for row in range(n_rows):
        rimages = images[row * images_per_row : (row + 1) * images_per_row]
        row_images.append(np.concatenate(rimages, axis=1))
    image = np.concatenate(row_images, axis=0)
    plt.imshow(image, cmap = mpl.cm.binary, **options)
    plt.axis("off")

plt.figure(figsize=(7, 4))
plt.subplot(121)
plot_digits(X_train[::2100])
plt.title("Original", fontsize=16)
plt.subplot(122)
plot_digits(X_recovered[::2100])
plt.title("Compressed", fontsize=16)
plt.show()

### 3.4 Kernel PCA (Non-linear)
We use the Swiss Roll dataset to demonstrate how Kernel PCA can unroll nonlinear data.

In [ ]:
from sklearn.datasets import make_swiss_roll
from sklearn.decomposition import KernelPCA

X_swiss, t = make_swiss_roll(n_samples=1000, noise=0.2, random_state=42)

rbf_pca = KernelPCA(n_components=2, kernel="rbf", gamma=0.04)
X_reduced = rbf_pca.fit_transform(X_swiss)

# Visualizing the unrolled Swiss Roll
plt.figure(figsize=(8, 6))
plt.scatter(X_reduced[:, 0], X_reduced[:, 1], c=t, cmap=plt.cm.hot)
plt.title("Kernel PCA (RBF) Unrolling")
plt.xlabel("z1")
plt.ylabel("z2")
plt.grid(True)
plt.show()

### 3.5 LLE (Locally Linear Embedding)
LLE works by first measuring how each training instance linearly relates to its closest neighbors (c), and then looking for a low-dimensional representation of the training set where these local relationships are best preserved.

In [ ]:
from sklearn.manifold import LocallyLinearEmbedding

lle = LocallyLinearEmbedding(n_components=2, n_neighbors=10, random_state=42)
X_reduced_lle = lle.fit_transform(X_swiss)

plt.figure(figsize=(8, 6))
plt.scatter(X_reduced_lle[:, 0], X_reduced_lle[:, 1], c=t, cmap=plt.cm.hot)
plt.title("Locally Linear Embedding (LLE)")
plt.xlabel("z1")
plt.ylabel("z2")
plt.grid(True)
plt.show()

## 4. Step-by-Step Explanation

### 1. Elbow Plot Analysis
**Input:** The cumulative sum of the explained variance ratio from standard PCA.
**Process:** As we add dimensions (x-axis), the total variance explained (y-axis) increases. It rises sharply at first and then flattens out.
**Output:** We see an "elbow" around 154 dimensions where the curve reaches 95%. This means the other 600+ pixels in the MNIST images are mostly noise or redundant border pixels and can be safely discarded.

### 2. Compression/Decompression
**Concept:** $X_{recovered} = X_{d-proj} \cdot V^T$
* When we reduce data, we lose information (compression loss).
* When we reconstruct it (`inverse_transform`), we map the 154 values back to 784 pixels.
* **Visual Check:** The compressed digits look slightly blurry (missing high-frequency details) but are perfectly recognizable. This proves PCA captured the structural essence of the digits.

### 3. Kernel PCA vs. LLE
* **Kernel PCA (RBF):** It used a similarity function to separate the rolled layers implicitly. The result is a 2D projection that looks somewhat unrolled.
* **LLE:** It looked at the neighbors of each point on the Swiss Roll. It noticed that points are flat locally. By preserving these neighbor distances while flattening, it successfully unrolled the Swiss Roll into a clean 2D strip, often better than PCA for manifolds.

## 5. Chapter Summary

* **Dimensionality Reduction** speeds up training, removes noise, and helps visualize data.
* **PCA:** The gold standard for linear projection. It rotates the data to align with the axes of highest variance.
* **SVD:** The linear algebra engine under PCA's hood.
* **Choosing Dimensions:** Use `n_components` as a float (e.g., 0.95) to preserve a % of variance, or look for the elbow in the cumulative variance plot.
* **Randomized PCA:** Use this for faster training on large datasets.
* **Kernel PCA:** Use this for complex nonlinear datasets (like spirals).
* **Manifold Learning (LLE, t-SNE):** Best for visualization and highly twisted datasets where simple projection destroys the structure.